In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [2]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='../dataset/mnist/',
                               train=True,
                               download=False, # True
                               transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist/',
                              train=False,
                              download=False, # 第一次下载True
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

## Implementation of Simple Residual Network

In [6]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
        self.conv1 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x+y)
    
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=5)
        self.mp = torch.nn.MaxPool2d(2)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.fc = torch.nn.Linear(512,10)
    
    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = Net()

In [7]:
criterion = torch.nn.CrossEntropyLoss() # 交叉熵损失
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #带有冲量，目的能冲过鞍面和部分局部最优

In [8]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # predicted是一行最大值的下标,dim=1表示沿着列的维度
            total += labels.size(0) # 当前batch里数据标签的数目，即batch_size
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [9]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.534
[1,   600] loss: 0.134
[1,   900] loss: 0.112
Accuracy on test set: 96 %
[2,   300] loss: 0.083
[2,   600] loss: 0.076
[2,   900] loss: 0.069
Accuracy on test set: 97 %
[3,   300] loss: 0.059
[3,   600] loss: 0.056
[3,   900] loss: 0.053
Accuracy on test set: 98 %
[4,   300] loss: 0.048
[4,   600] loss: 0.043
[4,   900] loss: 0.049
Accuracy on test set: 98 %
[5,   300] loss: 0.039
[5,   600] loss: 0.038
[5,   900] loss: 0.041
Accuracy on test set: 98 %
[6,   300] loss: 0.035
[6,   600] loss: 0.033
[6,   900] loss: 0.034
Accuracy on test set: 98 %
[7,   300] loss: 0.031
[7,   600] loss: 0.033
[7,   900] loss: 0.030
Accuracy on test set: 98 %
[8,   300] loss: 0.028
[8,   600] loss: 0.027
[8,   900] loss: 0.029
Accuracy on test set: 98 %
[9,   300] loss: 0.022
[9,   600] loss: 0.025
[9,   900] loss: 0.026
Accuracy on test set: 98 %
[10,   300] loss: 0.019
[10,   600] loss: 0.022
[10,   900] loss: 0.024
Accuracy on test set: 98 %


## GoogleNet - Inception Module

In [10]:
class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size=1)
    
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(1408, 10)
    
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x
    
model = Net()

In [11]:
criterion = torch.nn.CrossEntropyLoss() # 交叉熵损失
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #带有冲量，目的能冲过鞍面和部分局部最优

In [12]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.828
[1,   600] loss: 0.222
[1,   900] loss: 0.154
Accuracy on test set: 96 %
[2,   300] loss: 0.118
[2,   600] loss: 0.103
[2,   900] loss: 0.094
Accuracy on test set: 97 %
[3,   300] loss: 0.082
[3,   600] loss: 0.076
[3,   900] loss: 0.072
Accuracy on test set: 98 %
[4,   300] loss: 0.065
[4,   600] loss: 0.066
[4,   900] loss: 0.062
Accuracy on test set: 98 %
[5,   300] loss: 0.058
[5,   600] loss: 0.057
[5,   900] loss: 0.052
Accuracy on test set: 98 %
[6,   300] loss: 0.047
[6,   600] loss: 0.050
[6,   900] loss: 0.055
Accuracy on test set: 98 %
[7,   300] loss: 0.044
[7,   600] loss: 0.047
[7,   900] loss: 0.045
Accuracy on test set: 98 %
[8,   300] loss: 0.042
[8,   600] loss: 0.040
[8,   900] loss: 0.044
Accuracy on test set: 98 %
[9,   300] loss: 0.039
[9,   600] loss: 0.040
[9,   900] loss: 0.037
Accuracy on test set: 98 %
[10,   300] loss: 0.035
[10,   600] loss: 0.034
[10,   900] loss: 0.037
Accuracy on test set: 98 %
